## Function to create geojson file using 'osmxtract'

In [6]:
import json
from osmxtract import overpass, location
import geopandas as gpd


class TileCollection():
    
    def __init__(self, place, buffer_size, tag, values):
        self.place = place
        self.buffer_size = buffer_size  #-- buffer size in (Km)
        self.tag = tag
        self.values = values            #-- values must be a list (e.g) ['runway', taxiway]
     
    #====================================================
    # Define area in lat/long where tiles are extracted
    #====================================================
    def boudaries(self):     
        lat, lon = location.geocode(self.place)
        bound = location.from_buffer(lat, lon, buffer_size = self.buffer_size)  # Buffer size in meter(M) unit
        return bound

    
    #====================================================
    # Extracted featured tiles and save as a json file
    #===================================================
    def map_features_json_response(self): 
        bound = self.boudaries()
        query = overpass.ql_query(bound, tag = self.tag, values = self.values)
        response = overpass.request(query)
        return response
    

   #====================================================
    # Save as a goejson file
    #===================================================
    def json_to_geojson(self, file_name):
        self.file_name = file_name
        
        features_to_points = []
        my_data = self.map_features_json_response()
        
        for each_feature in my_data['elements']: 
            if each_feature['type'] == 'node':                                # 'node' element
                features_to_points = features_to_points + [each_feature]
        
            elif each_feature['type'] == 'way' :                              # 'way' element
                LAT = [coordinate['lat'] for coordinate in each_feature['geometry']]
                LON = [coordinate['lon'] for coordinate in each_feature['geometry']]
                TYPE = ['node'] * len(LAT)
                ID = [each_feature['id']] * len(LAT)
                TAGS = [each_feature['tags']] * len(LAT)
    
                points = list(zip(TYPE, ID, LAT, LON, TAGS))   #-- parsing to points
                parsed_to_points = [{'type':point[0], 'id':point[1], 'lat':point[2], 'lon':point[3], 
                             'tags':point[4]} for point in points]
                features_to_points = features_to_points + parsed_to_points 
        
            else:                                                             # 'relation' element
                parsed_relation = []
                index =  each_feature['id']
                tag = each_feature['tags']
                for member in each_feature['members']:
                    LAT = [coordinate['lat'] for coordinate in member['geometry']]
                    LON = [coordinate['lon'] for coordinate in member['geometry']]
                    TYPE = ['node'] * len(LAT)  
                    ID = [index] * len(LAT)
                    TAGS = [tag] * len(LAT)
                                    
                    points = list(zip(TYPE, ID, LAT, LON, TAGS))
                    parsed_to_way_to_point = [{'type':point[0], 'id':point[1], 'lat':point[2], 'lon':point[3], 
                                             'tags':point[4]} for point in points]
            
                parsed_relation = parsed_relation + parsed_to_way_to_point
                features_to_points = features_to_points + parsed_relation  
                
        my_data['elements'] = features_to_points
        feature_collection = overpass.as_geojson(my_data, 'point')
        
        f_name = file_name + '.geojson'
        with open(f_name, 'w') as f:
            json.dump(feature_collection, f)
        
        return         

### TileCollection( )
#### A function _TileCollection()_ acquires four parameters
* **place** - specify the area where maptiles are collected
* **buffer_size** - specify buffer size in kilometer from the place 
* **tag** - a tag value of OSM map features (https://wiki.openstreetmap.org/wiki/Map_Feature)
* **values** - values of OSM map feature that belong(s) to a tag (https://wiki.openstreetmap.org/wiki/Map_Feature)

#### 1. Create an object with 4 parameters to collect airfields tiles from the area around 200 km of Beijing, China

In [2]:
CN_mil = TileCollection(place = "Beijing, China", buffer_size = 200000, tag='military', values=['airfield'])

#### 2. TileCollection( ).boundaries( )
Returns the area boundaries in latitude and longitude. No parameter required.

In [3]:
CN_mil.boudaries()

(38.24450916005233, 114.14625235383501, 41.827710681937326, 118.86280948780546)

#### 3. TileCollection( ).map_features_json_response( )
Create map tiles to json file. No parameter required.

In [4]:
CN_mil.map_features_json_response()

{'version': 0.6,
 'generator': 'Overpass API 0.7.55.7 8b86ff77',
 'osm3s': {'timestamp_osm_base': '2019-12-04T17:27:02Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'way',
   'id': 563938445,
   'bounds': {'minlat': 38.7710902,
    'minlon': 117.0530777,
    'maxlat': 38.7941876,
    'maxlon': 117.0753719},
   'nodes': [5434108694,
    5434108695,
    5434108696,
    5434108697,
    5434108698,
    1668287601,
    1668287597,
    1668287580,
    1668287583,
    1668287773,
    1668287783,
    1668287793,
    1668287789,
    1668287769,
    1668287753,
    5434108699,
    5434108700,
    5434108701,
    5434108702,
    5434108703,
    5434108704,
    5434108705,
    5434108706,
    5434108707,
    5434108708,
    5434108709,
    5434108710,
    5434108711,
    5434108712,
    5434108713,
    5434108714,
    5434108715,
    5434108716,
    5434108717,
    5434108718,
    5434108694],
 

#### 4. TileCollection().json_to_geojson( _file_name_ )
Create geojson file. A file name required (file extension 'geojson' is not necessary.)

In [5]:
CN_mil.json_to_geojson('airfield_Bejing')